In [1]:
import pandas as pd
import requests

## Data exploration

Load the members and jobs data from the endpoints for investigation

In [2]:
# Define the URL
url = "https://bn-hiring-challenge.fly.dev/members.json"

# Fetch the JSON data from the URL
response = requests.get(url)
data = response.json()

# Load the JSON data into a DataFrame
members_df = pd.DataFrame(data)
members_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    5 non-null      object
 1   bio     5 non-null      object
dtypes: object(2)
memory usage: 212.0+ bytes


In [3]:
members_df.head()

,name,bio
0,Joe,"I'm a designer from London, UK"
1,Marta,I'm looking for an internship in London
2,Hassan,I'm looking for a design job
3,Grace,I'm looking for a job in marketing outside of ...
4,Daisy,I'm a software developer currently in Edinburg...


In [4]:
# Define the URL
url = "https://bn-hiring-challenge.fly.dev/jobs.json"

# Fetch the JSON data from the URL
response = requests.get(url)
data = response.json()

# Load the JSON data into a DataFrame
jobs_df = pd.DataFrame(data)
jobs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     8 non-null      object
 1   location  8 non-null      object
dtypes: object(2)
memory usage: 260.0+ bytes


In [5]:
jobs_df.head()

,title,location
0,Software Developer,London
1,Marketing Internship,York
2,Data Scientist,London
3,Legal Internship,London
4,Project Manager,Manchester


## Conclusions

We have a set of members who have a `name` a `role` and an optional `location` preference. We have a set of jobs with a `role` (title) and a `location`.

As a data science exercise there could be some additional work here to normalise entities, such as job roles and locations and build out some kind of similarity matrix and distance searches.

For the purposes of this exercise, we'll just use string manipulation initially.

## Example Recommendation Process

This initial recommendation process will use a naive string-matching approach

In [55]:
# Function to determine location preference from bio
def get_location(bio):
    bio = bio.lower()

    if 'outside of london' in bio:
        return 'outside of London'
    elif 'london' in bio:
        return 'London'

    return None

In [56]:
# Function to match job title with member bio
def match_job_title(bio, job_title):
    bio = bio.lower()
    job_title = job_title.lower()
    if 'design' in bio and 'design' in job_title:
        return True
    if 'internship' in bio and 'internship' in job_title:
        return True
    if 'software developer' in bio and 'software developer' in job_title:
        return True
    if 'marketing' in bio and 'marketing' in job_title:
        return True
    return False

In [57]:
# Function to get recommendations for a single member
def recommend_for_member(member):
    member_bio = member['bio']
    member_location = member['location']
    
    def job_match(job):
        job_location = job['location']
        job_title = job['title']
        
        # Match job location with member's location preference
        if member_location == 'outside of London' and job_location == 'London':
            return False
        if member_location and member_location != 'outside of London' and job_location != member_location:
            return False
        
        # Match job title with member's bio
        return match_job_title(member_bio, job_title)
    
    # Filter jobs that match the member's preferences
    matched_jobs = jobs_df[jobs_df.apply(job_match, axis=1)]
    return matched_jobs.to_dict('records')

In [58]:
# Add location preferences to members dataframe
members_df['location'] = members_df['bio'].apply(get_location)

In [59]:
members_df

,name,bio,location
0,Joe,"I'm a designer from London, UK",London
1,Marta,I'm looking for an internship in London,London
2,Hassan,I'm looking for a design job,None
3,Grace,I'm looking for a job in marketing outside of ...,outside of London
4,Daisy,I'm a software developer currently in Edinburg...,London


In [60]:
# Create the recommendations dictionary
recommendations = {member['name']: recommend_for_member(member) for _, member in members_df.iterrows()}

recommendations

{'Joe': [{'title': 'UX Designer', 'location': 'London'}],
 'Marta': [{'title': 'Legal Internship', 'location': 'London'},
  {'title': 'Sales Internship', 'location': 'London'}],
 'Hassan': [{'title': 'UX Designer', 'location': 'London'}],
 'Grace': [{'title': 'Marketing Internship', 'location': 'York'}],
 'Daisy': [{'title': 'Software Developer', 'location': 'London'}]}

In [61]:
# Print recommendations
for member, jobs in recommendations.items():
    print(f"{member}:")
    for job in jobs:
        print(f"  - {job['title']} in {job['location']}")

Joe:
  - UX Designer in London
Marta:
  - Legal Internship in London
  - Sales Internship in London
Hassan:
  - UX Designer in London
Grace:
  - Marketing Internship in York
Daisy:
  - Software Developer in London
